In [46]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [47]:
@dataclass 
class Message:
    content : str

In [48]:
class SimpleAgent(RoutedAgent):
    def __init__(self)->None:
        super().__init__("Simple")

    @message_handler
    async def on_my_message(self, message : Message, ctx: MessageContext) -> Message:
        return Message(content = f"This is {self.id.type}-{self.id.key}. You said '{message.content}' and I disagee.")

Creating a Single Threaded Standalone Runtime

In [49]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

In [50]:
runtime.start()

In [51]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well, hi there!"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said 'Well, hi there!' and I disagee.


In [52]:
await runtime.stop()
await runtime.close()

Using AgentChat Assistant

In [53]:
class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)

In [54]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [55]:
runtime.start()
response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

LLMAgent received message: Hi there!
LLMAgent responded: Hello! How can I assist you today?
>>> Hello! How can I assist you today?
>>> This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagee.
LLMAgent received message: This is simple_agent-default. You said 'Hello! How can I assist you today?' and I disagee.
LLMAgent responded: I apologize if my response didn't meet your expectations. How can I assist you better?


In [56]:
await runtime.stop()
await runtime.close()